## Importing important libraries

In [ ]:
import os
import random
import uuid
import shutil
from PIL import Image
import torchvision.transforms.functional as F

## Importing Dataset

In [2]:
dataset_path = '/kaggle/working/'

## Applying Augmentation

In [ ]:
original_data_dir = '/kaggle/input/chase-dataset/chase/images/'
augmented_data_dir = '/kaggle/working/augmented_images/'
original_mask_dir = '/kaggle/input/chase-dataset/chase/targets_1/'
augmented_mask_dir = '/kaggle/working/augmented_masks/'
os.makedirs(augmented_data_dir, exist_ok=True)
os.makedirs(augmented_mask_dir, exist_ok=True)

In [ ]:
class PairedTransform:
    def __init__(self, degrees=(0, 360), contrast=(1.0, 1.5), p_flip=0.5):
        self.degrees = degrees
        self.contrast = contrast
        self.p_flip = p_flip

    def __call__(self, image, mask,deg):
        # 1. Rotate with the same random angle
        angle = random.uniform(deg,deg)
        image = F.rotate(image, angle)
        mask = F.rotate(mask, angle)

        # 2. Random horizontal flip
        if random.random() < self.p_flip:
            image = F.hflip(image)
            mask = F.hflip(mask)

        # 3. Random vertical flip
        if random.random() < self.p_flip:
            image = F.vflip(image)
            mask = F.vflip(mask)

        # 4. Color jitter (only apply to image, not mask)
        contrast_factor = random.uniform(*self.contrast)
        image = F.adjust_contrast(image, contrast_factor)

        return image, mask

augment = PairedTransform()


In [ ]:
# Process each image in the directory
for img_file in os.listdir(original_data_dir):
    image_path = os.path.join(original_data_dir, img_file)
    mask_path = os.path.join(original_mask_dir, img_file.replace(".jpg", "_1stHO.png"))  # Adjust mask naming convention
    
    # Load image and mask
    image = Image.open(image_path).convert("RGB")
    mask = Image.open(mask_path).convert("L")  # Convert mask to single channel (grayscale)
    
    # Generate 5 augmented versions for each image-mask pair
    for i in range(5):
        augmented_image, augmented_mask = augment(image, mask,72*i)
        
        # Save augmented image and mask with unique filenames
        unique_filename = uuid.uuid4()
        augmented_image.save(os.path.join(augmented_data_dir, f"{unique_filename}.png"))
        augmented_mask.save(os.path.join(augmented_mask_dir, f"{unique_filename}_1stHO.png"))


## Saving in zip format

In [ ]:
# Paths for folders to be zipped
augmented_images_dir = '/kaggle/working/augmented_images'
augmented_masks_dir = '/kaggle/working/augmented_masks'

# Create zip files for each folder
shutil.make_archive(augmented_images_dir, 'zip', augmented_images_dir)
shutil.make_archive(augmented_masks_dir, 'zip', augmented_masks_dir)

# File paths for the zip files
augmented_images_zip_path = augmented_images_dir + '.zip'
augmented_masks_zip_path = augmented_masks_dir + '.zip'

# Check if the zip files are created and then print their paths for download
if os.path.exists(augmented_images_zip_path):
    print(f"Augmented images zip file created at: {augmented_images_zip_path}")
if os.path.exists(augmented_masks_zip_path):
    print(f"Augmented masks zip file created at: {augmented_masks_zip_path}")

Preprocessed images zip file created at: /kaggle/working/preprocessed.zip
Augmented images zip file created at: /kaggle/working/augmented_images.zip
Augmented masks zip file created at: /kaggle/working/augmented_masks.zip
